In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'SVHN2.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_data']
    train_labels = save['train_labels']
    valid_dataset = save['valid_data']
    valid_labels = save['valid_labels']
    test_dataset = save['test_data']
    test_labels = save['test_labels']
    test_filenames = save['test_filenames']
    del save  # hint to help gc free up memory

    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (225630, 32, 32, 1) (225630, 5)
Validation set (10000, 32, 32, 1) (10000, 5)
Test set (13066, 32, 32, 1) (13066, 5)


In [3]:
def LecunLCN(X, image_shape, threshold=1e-4, radius=7, use_divisor=True):
    """Local Contrast Normalization"""
    """[http://yann.lecun.com/exdb/publis/pdf/jarrett-iccv-09.pdf]"""

    # Get Gaussian filter
    filter_shape = (radius, radius, image_shape[3], 1)

    #self.filters = theano.shared(self.gaussian_filter(filter_shape), borrow=True)
    filters = gaussian_filter(filter_shape)
    X = tf.convert_to_tensor(X, dtype=tf.float32)
    # Compute the Guassian weighted average by means of convolution
    convout = tf.nn.conv2d(X, filters, [1,1,1,1], 'SAME')

    # Subtractive step
    mid = int(np.floor(filter_shape[1] / 2.))

    # Make filter dimension broadcastable and subtract
    centered_X = tf.sub(X, convout)

    # Boolean marks whether or not to perform divisive step
    if use_divisor:
        # Note that the local variances can be computed by using the centered_X
        # tensor. If we convolve this with the mean filter, that should give us
        # the variance at each point. We simply take the square root to get our
        # denominator

        # Compute variances
        sum_sqr_XX = tf.nn.conv2d(tf.square(centered_X), filters, [1,1,1,1], 'SAME')

        # Take square root to get local standard deviation
        denom = tf.sqrt(sum_sqr_XX)

        per_img_mean = tf.reduce_mean(denom)
        divisor = tf.maximum(per_img_mean, denom)
        # Divisise step
        new_X = tf.truediv(centered_X, tf.maximum(divisor, threshold))
    else:
        new_X = centered_X

    return new_X


def gaussian_filter(kernel_shape):
    x = np.zeros(kernel_shape, dtype = float)
    mid = np.floor(kernel_shape[0] / 2.)
    
    for kernel_idx in xrange(0, kernel_shape[2]):
        for i in xrange(0, kernel_shape[0]):
            for j in xrange(0, kernel_shape[1]):
                x[i, j, kernel_idx, 0] = gauss(i - mid, j - mid)
    
    return tf.convert_to_tensor(x / np.sum(x), dtype=tf.float32)

def gauss(x, y, sigma=3.0):
    Z = 2 * np.pi * sigma ** 2
    return  1. / Z * np.exp(-(x ** 2 + y ** 2) / (2. * sigma ** 2))

In [4]:
def accuracy(predictions, labels, printstat=False):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [5]:
image_size = 32
num_labels = 11 # 10 + invalid
num_channels = 1 # grayscale

batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 512
num_hidden2 = 128
shape=[batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout
# F6: fully-connected layer, weight size: 64 x 11
# Output layer, weight size: 16 x 11

train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

    # Input data placeholders.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 5))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    
    layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    
    layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')
    
    layer4_weights_d1 = tf.get_variable("W4d1", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d1')
    layer5_weights_d1 = tf.get_variable("W5d1", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d1')

    layer4_weights_d2 = tf.get_variable("W4d2", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d2')
    layer5_weights_d2 = tf.get_variable("W5d2", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d2')
    
    layer4_weights_d3 = tf.get_variable("W4d3", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d3')
    layer5_weights_d3 = tf.get_variable("W5d3", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d3')

    layer4_weights_d4 = tf.get_variable("W4d4", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d4')
    layer5_weights_d4 = tf.get_variable("W5d4", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d4')
    
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P2')
        
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P4')
        
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')        
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        #print(reshape.get_shape())
        # hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        hidden1 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d1) + layer4_biases_d1)
        hidden2 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d2) + layer4_biases_d2)
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d3) + layer4_biases_d3)
        hidden4 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d4) + layer4_biases_d4)
        
        logits1 = tf.matmul(hidden1, layer5_weights_d1) + layer5_biases_d1
        logits2 = tf.matmul(hidden2, layer5_weights_d2) + layer5_biases_d2
        logits3 = tf.matmul(hidden3, layer5_weights_d3) + layer5_biases_d3
        logits4 = tf.matmul(hidden4, layer5_weights_d4) + layer5_biases_d4
        
        return [logits1, logits2, logits3, logits4]
    
    # Training computation.
    [logits1, logits2, logits3, logits4] = model(tf_train_dataset, 0.6, shape)
    
    # L2 regularization for the fully connected parameters.
    regularizers = [(tf.nn.l2_loss(layer5_weights_d1) + tf.nn.l2_loss(layer5_biases_d1)),
                    (tf.nn.l2_loss(layer5_weights_d2) + tf.nn.l2_loss(layer5_biases_d2)),
                    (tf.nn.l2_loss(layer5_weights_d3) + tf.nn.l2_loss(layer5_biases_d3)),
                    (tf.nn.l2_loss(layer5_weights_d4) + tf.nn.l2_loss(layer5_biases_d4))]
    
    post_logits = [tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4]))]
    
    loss = (post_logits[0] + 1e-4 * regularizers[0]) + (post_logits[1] + 1e-4 * regularizers[1]) + \
           (post_logits[2] + 1e-4 * regularizers[2]) + (post_logits[3] + 1e-4 * regularizers[3]) 
    
    # Optimizer.
    batch = tf.Variable(0, dtype=tf.float32)
    
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(0.01, batch * batch_size, train_size, 0.95, staircase=True)
    
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    
    # Predictions for the training, validation and test data.
    train_logits = model(tf_train_dataset, 1.0, shape)
    train_prediction = tf.pack([tf.nn.softmax(train_logits[0]), tf.nn.softmax(train_logits[1]),\
                                tf.nn.softmax(train_logits[2]), tf.nn.softmax(train_logits[3])])
    
    valid_logits = model(tf_valid_dataset, 1.0, shape)
    valid_prediction = tf.pack([tf.nn.softmax(valid_logits[0]), tf.nn.softmax(valid_logits[1]),\
                                tf.nn.softmax(valid_logits[2]), tf.nn.softmax(valid_logits[3])])
    
    test_logits = model(tf_test_dataset, 1.0, shape)
    test_prediction = tf.pack([tf.nn.softmax(test_logits[0]), tf.nn.softmax(test_logits[1]),\
                               tf.nn.softmax(test_logits[2]), tf.nn.softmax(test_logits[3]),])

    saver = tf.train.Saver()

In [6]:
num_steps = 25001

loss_values = []
train_accuracies = []
validation_accuracies = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        #if step % 10 == 0:
        #    loss_values.append(l)
            
        if (step % 5000 == 0): 
            train_accuracy = accuracy(predictions, batch_labels[:,1:5])
            valid_accuracy = accuracy(valid_prediction.eval(), valid_labels[:,1:5])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)    
            print('Validation accuracy: %.1f%%' % valid_accuracy)
            train_accuracies.append(train_accuracy)
            validation_accuracies.append(valid_accuracy)
            
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:5]))
    
    save_path = saver.save(session, "ConvNet2")
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 15.410473
Minibatch accuracy: 28.1%
Validation accuracy: 38.0%
Minibatch loss at step 5000: 1.913260
Minibatch accuracy: 89.1%
Validation accuracy: 92.5%
Minibatch loss at step 10000: 0.875781
Minibatch accuracy: 95.3%
Validation accuracy: 94.7%
Minibatch loss at step 15000: 0.798683
Minibatch accuracy: 94.5%
Validation accuracy: 95.4%
Minibatch loss at step 20000: 0.447861
Minibatch accuracy: 97.7%
Validation accuracy: 95.7%
Minibatch loss at step 25000: 0.626380
Minibatch accuracy: 96.1%
Validation accuracy: 96.1%
Test accuracy: 93.9%
Model saved in file: ConvNet2


In [7]:
num_steps = 300001

loss_values = []
train_accuracies = []
validation_accuracies = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        #if step % 10 == 0:
        #    loss_values.append(l)
            
        if (step % 10000 == 0): 
            train_accuracy = accuracy(predictions, batch_labels[:,1:5])
            valid_accuracy = accuracy(valid_prediction.eval(), valid_labels[:,1:5])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)    
            print('Validation accuracy: %.1f%%' % valid_accuracy)
            train_accuracies.append(train_accuracy)
            validation_accuracies.append(valid_accuracy)
            
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:5]))
    
    save_path = saver.save(session, "ConvNet2")
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 14.380356
Minibatch accuracy: 12.5%
Validation accuracy: 33.2%
Minibatch loss at step 10000: 1.116303
Minibatch accuracy: 94.5%
Validation accuracy: 94.6%
Minibatch loss at step 20000: 0.437425
Minibatch accuracy: 98.4%
Validation accuracy: 95.6%
Minibatch loss at step 30000: 0.631603
Minibatch accuracy: 97.7%
Validation accuracy: 96.0%
Minibatch loss at step 40000: 0.847581
Minibatch accuracy: 97.3%
Validation accuracy: 96.3%
Minibatch loss at step 50000: 0.360818
Minibatch accuracy: 98.8%
Validation accuracy: 96.3%
Minibatch loss at step 60000: 0.499509
Minibatch accuracy: 97.7%
Validation accuracy: 96.4%
Minibatch loss at step 70000: 0.598947
Minibatch accuracy: 97.7%
Validation accuracy: 96.4%
Minibatch loss at step 80000: 0.585644
Minibatch accuracy: 98.4%
Validation accuracy: 96.5%
Minibatch loss at step 90000: 0.513776
Minibatch accuracy: 96.9%
Validation accuracy: 96.5%
Minibatch loss at step 100000: 0.418904
Minibatch accuracy: 97.7%
Valid

### Model Validation

In [8]:
from sklearn.utils import shuffle

train_size = train_labels.shape[0]
valid_size = valid_labels.shape[0]
test_size = test_labels.shape[0]

dataset_tmp = np.concatenate((train_dataset, valid_dataset,test_dataset))
labels_tmp = np.concatenate((train_labels, valid_labels, test_labels))
dataset, labels = shuffle(dataset_tmp, labels_tmp)

train_dataset = dataset[:train_size,:,:,:]
dataset = np.delete(dataset, np.arange(train_size), axis=0)
train_labels = labels[:train_size,:]
labels = np.delete(labels, np.arange(train_size), axis=0)

test_dataset = dataset[:test_size,:,:,:]
dataset = np.delete(dataset, np.arange(test_size), axis=0)
test_labels = labels[:test_size,:]
labels = np.delete(labels, np.arange(test_size), axis=0)

valid_dataset = dataset
valid_labels = labels

In [9]:
image_size = 32
num_labels = 11 # 10 + invalid
num_channels = 1 # grayscale

batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 512
num_hidden2 = 128
shape=[batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout
# F6: fully-connected layer, weight size: 64 x 11
# Output layer, weight size: 16 x 11

train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

    # Input data placeholders.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 5))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    
    layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    
    layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')
    
    layer4_weights_d1 = tf.get_variable("W4d1", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d1')
    layer5_weights_d1 = tf.get_variable("W5d1", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d1')

    layer4_weights_d2 = tf.get_variable("W4d2", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d2')
    layer5_weights_d2 = tf.get_variable("W5d2", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d2')
    
    layer4_weights_d3 = tf.get_variable("W4d3", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d3')
    layer5_weights_d3 = tf.get_variable("W5d3", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d3')

    layer4_weights_d4 = tf.get_variable("W4d4", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d4')
    layer5_weights_d4 = tf.get_variable("W5d4", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d4')
    
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P2')
        
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P4')
        
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')        
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        #print(reshape.get_shape())
        # hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        hidden1 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d1) + layer4_biases_d1)
        hidden2 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d2) + layer4_biases_d2)
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d3) + layer4_biases_d3)
        hidden4 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d4) + layer4_biases_d4)
        
        logits1 = tf.matmul(hidden1, layer5_weights_d1) + layer5_biases_d1
        logits2 = tf.matmul(hidden2, layer5_weights_d2) + layer5_biases_d2
        logits3 = tf.matmul(hidden3, layer5_weights_d3) + layer5_biases_d3
        logits4 = tf.matmul(hidden4, layer5_weights_d4) + layer5_biases_d4
        
        return [logits1, logits2, logits3, logits4]
    
    # Training computation.
    [logits1, logits2, logits3, logits4] = model(tf_train_dataset, 0.6, shape)
    
    # L2 regularization for the fully connected parameters.
    regularizers = [(tf.nn.l2_loss(layer5_weights_d1) + tf.nn.l2_loss(layer5_biases_d1)),
                    (tf.nn.l2_loss(layer5_weights_d2) + tf.nn.l2_loss(layer5_biases_d2)),
                    (tf.nn.l2_loss(layer5_weights_d3) + tf.nn.l2_loss(layer5_biases_d3)),
                    (tf.nn.l2_loss(layer5_weights_d4) + tf.nn.l2_loss(layer5_biases_d4))]
    
    post_logits = [tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4]))]
    
    loss = (post_logits[0] + 1e-4 * regularizers[0]) + (post_logits[1] + 1e-4 * regularizers[1]) + \
           (post_logits[2] + 1e-4 * regularizers[2]) + (post_logits[3] + 1e-4 * regularizers[3]) 
    
    # Optimizer.
    batch = tf.Variable(0, dtype=tf.float32)
    
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(0.01, batch * batch_size, train_size, 0.95, staircase=True)
    
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    
    # Predictions for the training, validation and test data.
    train_logits = model(tf_train_dataset, 1.0, shape)
    train_prediction = tf.pack([tf.nn.softmax(train_logits[0]), tf.nn.softmax(train_logits[1]),\
                                tf.nn.softmax(train_logits[2]), tf.nn.softmax(train_logits[3])])
    
    valid_logits = model(tf_valid_dataset, 1.0, shape)
    valid_prediction = tf.pack([tf.nn.softmax(valid_logits[0]), tf.nn.softmax(valid_logits[1]),\
                                tf.nn.softmax(valid_logits[2]), tf.nn.softmax(valid_logits[3])])
    
    test_logits = model(tf_test_dataset, 1.0, shape)
    test_prediction = tf.pack([tf.nn.softmax(test_logits[0]), tf.nn.softmax(test_logits[1]),\
                               tf.nn.softmax(test_logits[2]), tf.nn.softmax(test_logits[3]),])

    saver = tf.train.Saver()

In [10]:
num_steps = 300001

loss_values = []
train_accuracies = []
validation_accuracies = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        #if step % 10 == 0:
        #    loss_values.append(l)
            
        if (step % 10000 == 0): 
            train_accuracy = accuracy(predictions, batch_labels[:,1:5])
            valid_accuracy = accuracy(valid_prediction.eval(), valid_labels[:,1:5])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)    
            print('Validation accuracy: %.1f%%' % valid_accuracy)
            train_accuracies.append(train_accuracy)
            validation_accuracies.append(valid_accuracy)
            
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:5]))
    
    save_path = saver.save(session, "ConvNet2")
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 17.572220
Minibatch accuracy: 5.5%
Validation accuracy: 44.4%
Minibatch loss at step 10000: 1.476254
Minibatch accuracy: 91.8%
Validation accuracy: 94.6%
Minibatch loss at step 20000: 0.516154
Minibatch accuracy: 97.7%
Validation accuracy: 95.5%
Minibatch loss at step 30000: 0.541174
Minibatch accuracy: 97.3%
Validation accuracy: 95.8%
Minibatch loss at step 40000: 0.472113
Minibatch accuracy: 98.0%
Validation accuracy: 96.1%
Minibatch loss at step 50000: 0.415869
Minibatch accuracy: 98.4%
Validation accuracy: 96.2%
Minibatch loss at step 60000: 0.203837
Minibatch accuracy: 99.6%
Validation accuracy: 96.2%
Minibatch loss at step 70000: 0.239876
Minibatch accuracy: 98.8%
Validation accuracy: 96.2%
Minibatch loss at step 80000: 0.199157
Minibatch accuracy: 98.4%
Validation accuracy: 96.3%
Minibatch loss at step 90000: 0.401924
Minibatch accuracy: 98.0%
Validation accuracy: 96.3%
Minibatch loss at step 100000: 0.389858
Minibatch accuracy: 99.2%
Valida

### Model Validation II

In [11]:
from sklearn.utils import shuffle

train_size = train_labels.shape[0]
valid_size = valid_labels.shape[0]
test_size = test_labels.shape[0]

dataset_tmp = np.concatenate((train_dataset, valid_dataset,test_dataset))
labels_tmp = np.concatenate((train_labels, valid_labels, test_labels))
dataset, labels = shuffle(dataset_tmp, labels_tmp)

train_dataset = dataset[:train_size,:,:,:]
dataset = np.delete(dataset, np.arange(train_size), axis=0)
train_labels = labels[:train_size,:]
labels = np.delete(labels, np.arange(train_size), axis=0)

test_dataset = dataset[:test_size,:,:,:]
dataset = np.delete(dataset, np.arange(test_size), axis=0)
test_labels = labels[:test_size,:]
labels = np.delete(labels, np.arange(test_size), axis=0)

valid_dataset = dataset
valid_labels = labels

In [12]:
image_size = 32
num_labels = 11 # 10 + invalid
num_channels = 1 # grayscale

batch_size = 64
patch_size = 5
depth1 = 16
depth2 = 32
depth3 = 64
num_hidden1 = 512
num_hidden2 = 128
shape=[batch_size, image_size, image_size, num_channels]

# Construct a 7-layer CNN.
# C1: convolutional layer, batch_size x 28 x 28 x 16, convolution size: 5 x 5 x 1 x 16
# S2: sub-sampling layer, batch_size x 14 x 14 x 16
# C3: convolutional layer, batch_size x 10 x 10 x 32, convolution size: 5 x 5 x 16 x 32
# S4: sub-sampling layer, batch_size x 5 x 5 x 32
# C5: convolutional layer, batch_size x 1 x 1 x 64, convolution size: 5 x 5 x 32 x 64
# Dropout
# F6: fully-connected layer, weight size: 64 x 11
# Output layer, weight size: 16 x 11

train_size = train_labels.shape[0]

graph = tf.Graph()
with graph.as_default():

    # Input data placeholders.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 5))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    layer1_weights = tf.get_variable("W1", shape=[patch_size, patch_size, num_channels, depth1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer1_biases = tf.Variable(tf.constant(1.0, shape=[depth1]), name='B1')
    
    layer2_weights = tf.get_variable("W2", shape=[patch_size, patch_size, depth1, depth2],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]), name='B2')
    
    layer3_weights = tf.get_variable("W3", shape=[patch_size, patch_size, depth2, num_hidden1],\
                                     initializer=tf.contrib.layers.xavier_initializer_conv2d())
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]), name='B3')
    
    layer4_weights_d1 = tf.get_variable("W4d1", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d1')
    layer5_weights_d1 = tf.get_variable("W5d1", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d1 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d1')

    layer4_weights_d2 = tf.get_variable("W4d2", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d2')
    layer5_weights_d2 = tf.get_variable("W5d2", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d2 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d2')
    
    layer4_weights_d3 = tf.get_variable("W4d3", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d3')
    layer5_weights_d3 = tf.get_variable("W5d3", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d3 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d3')

    layer4_weights_d4 = tf.get_variable("W4d4", shape=[num_hidden1, num_hidden2],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer4_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_hidden2]), name='B4d4')
    layer5_weights_d4 = tf.get_variable("W5d4", shape=[num_hidden2, num_labels],\
           initializer=tf.contrib.layers.xavier_initializer())
    layer5_biases_d4 = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='B5d4')
    
    # Model.
    def model(data, keep_prob, shape):
        LCN = LecunLCN(data, shape)
        
        conv = tf.nn.conv2d(LCN, layer1_weights, [1,1,1,1], 'VALID', name='C1')
        hidden = tf.nn.relu(conv + layer1_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P2')
        
        conv = tf.nn.conv2d(sub, layer2_weights, [1,1,1,1], padding='VALID', name='C3')
        hidden = tf.nn.relu(conv + layer2_biases)
        lrn = tf.nn.local_response_normalization(hidden)
        sub = tf.nn.max_pool(lrn, [1,2,2,1], [1,2,2,1], 'SAME', name='P4')
        
        conv = tf.nn.conv2d(sub, layer3_weights, [1,1,1,1], padding='VALID', name='C5')        
        hidden = tf.nn.relu(conv + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        #print(reshape.get_shape())
        # hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
        hidden1 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d1) + layer4_biases_d1)
        hidden2 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d2) + layer4_biases_d2)
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d3) + layer4_biases_d3)
        hidden4 = tf.nn.relu(tf.matmul(reshape, layer4_weights_d4) + layer4_biases_d4)
        
        logits1 = tf.matmul(hidden1, layer5_weights_d1) + layer5_biases_d1
        logits2 = tf.matmul(hidden2, layer5_weights_d2) + layer5_biases_d2
        logits3 = tf.matmul(hidden3, layer5_weights_d3) + layer5_biases_d3
        logits4 = tf.matmul(hidden4, layer5_weights_d4) + layer5_biases_d4
        
        return [logits1, logits2, logits3, logits4]
    
    # Training computation.
    [logits1, logits2, logits3, logits4] = model(tf_train_dataset, 0.6, shape)
    
    # L2 regularization for the fully connected parameters.
    regularizers = [(tf.nn.l2_loss(layer5_weights_d1) + tf.nn.l2_loss(layer5_biases_d1)),
                    (tf.nn.l2_loss(layer5_weights_d2) + tf.nn.l2_loss(layer5_biases_d2)),
                    (tf.nn.l2_loss(layer5_weights_d3) + tf.nn.l2_loss(layer5_biases_d3)),
                    (tf.nn.l2_loss(layer5_weights_d4) + tf.nn.l2_loss(layer5_biases_d4))]
    
    post_logits = [tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits1, tf_train_labels[:,1])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits2, tf_train_labels[:,2])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits3, tf_train_labels[:,3])),
                   tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits4, tf_train_labels[:,4]))]
    
    loss = (post_logits[0] + 1e-4 * regularizers[0]) + (post_logits[1] + 1e-4 * regularizers[1]) + \
           (post_logits[2] + 1e-4 * regularizers[2]) + (post_logits[3] + 1e-4 * regularizers[3]) 
    
    # Optimizer.
    batch = tf.Variable(0, dtype=tf.float32)
    
    # Decay once per epoch, using an exponential schedule starting at 0.01.
    learning_rate = tf.train.exponential_decay(0.01, batch * batch_size, train_size, 0.95, staircase=True)
    
    # Use simple momentum for the optimization.
    optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)
    
    # Predictions for the training, validation and test data.
    train_logits = model(tf_train_dataset, 1.0, shape)
    train_prediction = tf.pack([tf.nn.softmax(train_logits[0]), tf.nn.softmax(train_logits[1]),\
                                tf.nn.softmax(train_logits[2]), tf.nn.softmax(train_logits[3])])
    
    valid_logits = model(tf_valid_dataset, 1.0, shape)
    valid_prediction = tf.pack([tf.nn.softmax(valid_logits[0]), tf.nn.softmax(valid_logits[1]),\
                                tf.nn.softmax(valid_logits[2]), tf.nn.softmax(valid_logits[3])])
    
    test_logits = model(tf_test_dataset, 1.0, shape)
    test_prediction = tf.pack([tf.nn.softmax(test_logits[0]), tf.nn.softmax(test_logits[1]),\
                               tf.nn.softmax(test_logits[2]), tf.nn.softmax(test_logits[3]),])

    saver = tf.train.Saver()

In [13]:
num_steps = 300001

loss_values = []
train_accuracies = []
validation_accuracies = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()  
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        #if step % 10 == 0:
        #    loss_values.append(l)
            
        if (step % 10000 == 0): 
            train_accuracy = accuracy(predictions, batch_labels[:,1:5])
            valid_accuracy = accuracy(valid_prediction.eval(), valid_labels[:,1:5])
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % train_accuracy)    
            print('Validation accuracy: %.1f%%' % valid_accuracy)
            train_accuracies.append(train_accuracy)
            validation_accuracies.append(valid_accuracy)
            
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels[:,1:5]))
    
    save_path = saver.save(session, "ConvNet2")
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 20.561945
Minibatch accuracy: 4.3%
Validation accuracy: 44.3%
Minibatch loss at step 10000: 1.317839
Minibatch accuracy: 91.0%
Validation accuracy: 94.4%
Minibatch loss at step 20000: 1.124110
Minibatch accuracy: 93.8%
Validation accuracy: 95.3%
Minibatch loss at step 30000: 0.672407
Minibatch accuracy: 96.5%
Validation accuracy: 95.8%
Minibatch loss at step 40000: 0.769540
Minibatch accuracy: 96.1%
Validation accuracy: 96.0%
Minibatch loss at step 50000: 0.632046
Minibatch accuracy: 96.5%
Validation accuracy: 96.0%
Minibatch loss at step 60000: 0.602638
Minibatch accuracy: 96.9%
Validation accuracy: 96.1%
Minibatch loss at step 70000: 0.397593
Minibatch accuracy: 98.0%
Validation accuracy: 96.1%
Minibatch loss at step 80000: 0.630657
Minibatch accuracy: 97.3%
Validation accuracy: 96.1%
Minibatch loss at step 90000: 0.581101
Minibatch accuracy: 98.0%
Validation accuracy: 96.2%
Minibatch loss at step 100000: 0.311550
Minibatch accuracy: 98.8%
Valida